二軸倒單擺計算(參考Acrobot)

以下為Acrobot的動態方程式
$$\begin{align*}
    &\begin{bmatrix}
    m_1 l_{c1}^2 + m_2(l_1^2 + l_{c2}^2 + 2 l_1 l_{c2} C_{q2}) + I_1 +I_2
    &&
    m_2(l_{c2}^2 + l_1 l_{c2} C_{q2} + I_2)
    \\
    m_2(l_{c2}^2 + l_1 l_{c2} C_{q2}) + I_2
    &&
    m_2 l_{c2}^2 + I_2
    \end{bmatrix}
    \begin{bmatrix}
    \ddot{q_1} \\
    \ddot{q_2}
    \end{bmatrix}
    \\
    +
    &\begin{bmatrix}
    -m_2 l_1 l_{c2} S_{q2} \dot{q}_2^2 - 2m_1 l_1 l_{c2} S_{q2} \dot{q}_1 \dot{q}_2
    \\
    m_2 l_1 l_{c2} S_{q2} \dot{q}_1^2
    \end{bmatrix}
    +
    \begin{bmatrix}
    (m_1 l_{c1} + m_2 l_1) g C_{q1} + m_2 l_{c2} g C_{q_1 + q_2}
    \\
    m_2 l_{c2} g C_{q_1 + q_2}
    \end{bmatrix}
    =
    \begin{bmatrix}
    \tau_1 \\
    \tau_2
    \end{bmatrix}
\end{align*}$$

In [1]:
from sympy import *
from sympy.physics.mechanics import *

In [2]:
q1, q2 = dynamicsymbols('q1 q2')
m1, m2, l1, l2, lc1, lc2, I1, I2, t, g = symbols('m1 m2 l1 l2 l_c1 l_c2 I1 I2 t g')
dq1 = diff(q1, t)
dq2 = diff(q2, t)
ddq1 = diff(dq1, t)
ddq2 = diff(dq2, t)

Displace Vector $$\begin{align*}
    \vec{r}_1=
    \begin{bmatrix}
    l_{c1} C_{q1} \\
    l_{c1} S_{q1}
    \end{bmatrix}
    ,~~~
    \vec{r}_2=
    \begin{bmatrix}
    l_{1} C_{q1} + l_{c2} C_{q1+q2}\\
    l_{1} S_{q1} + l_{c2} S_{q1+q2}
    \end{bmatrix}
\end{align*}$$

In [3]:
r1 = Matrix([[lc1*cos(q1)], [lc1*sin(q1)]])
r2 = Matrix([[l1*cos(q1) + lc2*cos(q1+q2)], [l1*sin(q1) + lc2*sin(q1+q2)]])

Velocity Vectors

$$\begin{align*}
    &\vec{v}_1 = \frac{d}{dt}\vec{r}_1
    =
    \begin{bmatrix}
    -l_{c1} S_{q1} \dot{q}_1 \\
     l_{c1} C_{q1} \dot{q}_1
    \end{bmatrix}
    \\
    &\vec{v}_2 = \frac{d}{dt}\vec{r}_2
    =
    \begin{bmatrix}
    -l_{1} S_{q1} \dot{q}_1 - l_{c2} S_{q1+q2} 
    (\dot{q}_1 + \dot{q}_2)\\
     l_{1} C_{q1} \dot{q}_1 + l_{c2} C_{q1+q2}
    (\dot{q}_1 + \dot{q}_2)
    \end{bmatrix}
\end{align*}$$

In [4]:
v1 = diff(r1, t)
v2 = diff(r2, t)
w1 = dq1
w2 = dq1 + dq2

Kinetic Energy
$$\begin{align*}
    K_e = \frac{1}{2} m_1 \vec{v}_1^T \vec{v}_1
    + \frac{1}{2} m_2 \vec{v}_2^T \vec{v}_2
    + \frac{1}{2} I_1 \vec{\omega}_1^T \vec{\omega}_1
    + \frac{1}{2} I_2 \vec{\omega}_2^T \vec{\omega}_2
\end{align*}$$
在Sympy中，要讓矩陣平方變成常數要用.dot，可以試著跑以下兩個例子來看差異<br>
<code>m1*v1.T.dot(v1)</code><br>
<code>m1*v1.T*v1</code>

In [5]:
Ke = simplify( m1*v1.T.dot(v1) + m2*v2.T.dot(v2) + I1*w1**2 + I2*w2**2  )/2

整理一下可得到
\begin{align*}
    K_e = 
    \frac{1}{2}m_1 l_{c1}^2 \dot{q}_1^2
    +
    \frac{1}{2}m_2[l_{1}^2 \dot{q}_1^2 + l_{c2}^2 (\dot{q}_1 + \dot{q}_2)^2] + m_2 l_{1} l_{c2} C_{q2}  (\dot{q}_1^2 + \dot{q}_1\dot{q}_2)
    +
    \frac{1}{2} I_1 \dot{q}_1^2
    +
    \frac{1}{2} I_2 (\dot{q}_1+\dot{q}_2)^2
\end{align*}

計算位能
$$\begin{align*}
    P_e = m_1 g l_{c1} S_{q1} + 
    m_2 g ( l_1 S_{q1} + l_{c2} S_{q_1+q_2} )
\end{align*}$$

In [6]:
Pe = m1*g*lc1*sin(q1) + m2*g*(l1*sin(q1) + lc2*sin(q1+q2))

計算Lagrangian $$\mathcal{L}=K_e - P_e$$

In [7]:
L = Ke - Pe

計算EOM
$$\frac{d}{dt} \frac{\partial \mathcal{L}}{\partial \dot{q}}-\frac{\partial \mathcal{L}}{\partial q}=0$$
for no control input

In [8]:
eqq1 = simplify(diff( diff(L, dq1), t ) - diff(L, q1) )
eqq2 = simplify(diff( diff(L, dq2), t ) - diff(L, q2) )

In [9]:
M_11 = simplify( diff(eqq1, ddq1) )
M_12 = simplify( diff(eqq1, ddq2) )
M_21 = simplify( diff(eqq2, ddq1) )
M_22 = simplify( diff(eqq2, ddq2) )
MassMatrix = Matrix([[M_11, M_12], [M_21, M_22]])

In [10]:
MassMatrix

Matrix([
[I1 + I2 + l_c1**2*m1 + m2*(l1**2 + 2*l1*l_c2*cos(q2(t)) + l_c2**2), I2 + l_c2*m2*(l1*cos(q2(t)) + l_c2)],
[                           I2 + l1*l_c2*m2*cos(q2(t)) + l_c2**2*m2,                     I2 + l_c2**2*m2]])

剩下的部分

In [11]:
Matrix([simplify(eqq1 - M_11*ddq1 - M_12*ddq2), simplify(eqq2 - M_21*ddq1 - M_22*ddq2)])

Matrix([
[g*l1*m2*cos(q1(t)) + g*l_c1*m1*cos(q1(t)) + g*l_c2*m2*cos(q1(t) + q2(t)) - 2*l1*l_c2*m2*sin(q2(t))*Derivative(q1(t), t)*Derivative(q2(t), t) - l1*l_c2*m2*sin(q2(t))*Derivative(q2(t), t)**2],
[                                                                                                                      l_c2*m2*(g*cos(q1(t) + q2(t)) + l1*sin(q2(t))*Derivative(q1(t), t)**2)]])


回顧EOM
$$\begin{align*}
    &\begin{bmatrix}
    m_1 l_{c1}^2 + m_2(l_1^2 + l_{c2}^2 + 2 l_1 l_{c2} C_{q2}) + I_1 +I_2
    &&
    m_2(l_{c2}^2 + l_1 l_{c2} C_{q2} + I_2)
    \\
    m_2(l_{c2}^2 + l_1 l_{c2} C_{q2}) + I_2
    &&
    m_2 l_{c2}^2 + I_2
    \end{bmatrix}
    \begin{bmatrix}
    \ddot{q_1} \\
    \ddot{q_2}
    \end{bmatrix}
    \\
    +
    &\begin{bmatrix}
    -m_2 l_1 l_{c2} S_{q2} \dot{q}_2^2 - 2m_1 l_1 l_{c2} S_{q2} \dot{q}_1 \dot{q}_2
    \\
    m_2 l_1 l_{c2} S_{q2} \dot{q}_1^2
    \end{bmatrix}
    +
    \begin{bmatrix}
    (m_1 l_{c1} + m_2 l_1) g C_{q1} + m_2 l_{c2} g C_{q_1 + q_2}
    \\
    m_2 l_{c2} g C_{q_1 + q_2}
    \end{bmatrix}
    =
    \begin{bmatrix}
    \tau_1 \\
    \tau_2
    \end{bmatrix}
\end{align*}$$
比對結果相同

In [12]:
lm = LagrangesMethod(L, [q1,q2])
lm.form_lagranges_equations()
RHS = lm.rhs()

In [13]:
RHS_simp = simplify(RHS)

In [14]:
lm.mass_matrix

Matrix([
[I1 + I2 + l_c1**2*m1 + m2*(2*l1**2 + 4*l1*l_c2*cos(q2(t)) + 2*l_c2**2)/2, I2 + m2*(2*l1*l_c2*cos(q2(t)) + 2*l_c2**2)/2],
[                            I2 + m2*(2*l1*l_c2*cos(q2(t)) + 2*l_c2**2)/2,                              I2 + l_c2**2*m2]])

In [15]:
MM = simplify(lm.mass_matrix)
q1s, q2s = symbols('q1s q2s')
MM = MM.replace(q1,q1s)
MM = MM.replace(q2,q2s)
octave_code(MM)

'[I1 + I2 + l_c1.^2.*m1 + m2.*(l1.^2 + 2*l1.*l_c2.*cos(q2s) + l_c2.^2) I2 + l_c2.*m2.*(l1.*cos(q2s) + l_c2); I2 + l_c2.*m2.*(l1.*cos(q2s) + l_c2) I2 + l_c2.^2.*m2]'

In [16]:
simplify(lm.mass_matrix)

Matrix([
[I1 + I2 + l_c1**2*m1 + m2*(l1**2 + 2*l1*l_c2*cos(q2(t)) + l_c2**2), I2 + l_c2*m2*(l1*cos(q2(t)) + l_c2)],
[                               I2 + l_c2*m2*(l1*cos(q2(t)) + l_c2),                     I2 + l_c2**2*m2]])

In [17]:
octave_code(RHS_simp[2,0])

'% Not supported in Octave:\n% Derivative\n% Derivative\n% q1\n% q2\n(-I2.*g.*l1.*m2.*cos(q1(t)) - I2.*g.*l_c1.*m1.*cos(q1(t)) + I2.*l1.*l_c2.*m2.*sin(q2(t)).*Derivative(q1(t), t).^2 + 2*I2.*l1.*l_c2.*m2.*sin(q2(t)).*Derivative(q1(t), t).*Derivative(q2(t), t) + I2.*l1.*l_c2.*m2.*sin(q2(t)).*Derivative(q2(t), t).^2 + g.*l1.*l_c2.^2.*m2.^2.*cos(q1(t) + 2*q2(t))/2 - g.*l1.*l_c2.^2.*m2.^2.*cos(q1(t))/2 - g.*l_c1.*l_c2.^2.*m1.*m2.*cos(q1(t)) + l1.^2.*l_c2.^2.*m2.^2.*sin(2*q2(t)).*Derivative(q1(t), t).^2/2 + l1.*l_c2.^3.*m2.^2.*sin(q2(t)).*Derivative(q1(t), t).^2 + 2*l1.*l_c2.^3.*m2.^2.*sin(q2(t)).*Derivative(q1(t), t).*Derivative(q2(t), t) + l1.*l_c2.^3.*m2.^2.*sin(q2(t)).*Derivative(q2(t), t).^2)./(I1.*I2 + I1.*l_c2.^2.*m2 + I2.*l1.^2.*m2 + I2.*l_c1.^2.*m1 + l1.^2.*l_c2.^2.*m2.^2.*sin(q2(t)).^2 + l_c1.^2.*l_c2.^2.*m1.*m2)'

In [1]:
import numpy as np
a = np.arange(15).reshape(3, 5)
a

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])